# Aplicación logística

 Robert May, en ```Simple mathematical models with very complicated dynamics``` propuso una aplicación matemática al estudiar sistemas no lineales en el área de biología, en el cuál se explica como esto podría extrapolarse a otras áreas, como la social o la económica. Mitchell Feigenbaum estudia este modelo, que hoy se conoce como aplicación logística.

 Supongamos que queremos modelar un una población. Partimos de una cantidad $x$ de individuos de la población y proponemos un coeficiente $r$ que represente la tasa combinada entre la mortalidad y la reproducción. Entonces la cantidad de individuos por unidad de tiempo para un instante dado sobre el que se tome ese coeficiente o tasa, podría estar dado por

$$ rx $$

 Por ejemplo, si el individuo estudiado es un conejo, y tenemos que $r = 3$ por unidad de tiempo anual, significaría que el modelo representa que la cantidad de conejos en una población se triplica anualmente.

 Ahora bien, esto significaría que para un año $n$, tenemos $ r^nx $ individuos, y vemos que crece exponencialmente. Entonces, podemos suponer un máximo teórico, y tener en cuenta las restricciones del entorno. Se propone entonces agregar un término:

$$ rx(1-x) $$

 Para esto, necesitamos que x sea porcentual y tenga dominio $x \in \left( 0,1 \right)$. Entonces cuando la población va creciendo, el termino $1-x$ tiende a cero, y nos sirve para limitar la población. Em general, para cuando llegue el próximo instante de tiempo ($n+1$), sobre el actual ($n$), nos queda:

$$ x_{n+1} = rx_n(1-x_n) $$

Pero esto también hace que si la población crece mucho en el nuevo termino, el otro lo hago bajar, formando así ciclo retroactivo.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def preparar_f(r):
    xn=np.arange(-10,10,0.1)
    xnplus1=r*x*(1-x)
    plt.plot(xn,xnplus1)
    plt.show()

widgets.interact(preparar_f,
                 r=widgets.IntSlider(min=1, max=200, step=5, value=1, description='Tasa de natalidad y mortalidad'))

interactive(children=(IntSlider(value=1, description='Tasa de natalidad y mortalidad', max=200, min=1, step=5)…

<function __main__.preparar_f(r)>

Vamos a ver varios términos de la sucesión, suponiendo que el comportamiento es anual, $r = 2$ y que nos interesa el comportamiento a largo plazo, para una población que tiene actualmente el 40% de su máximo teórico:

$$x_1 = 2*0.4*(1-0.4) = 0.48$$
$$x_2 = 2*x_1*(1-x_1) = 0.4992$$
$$x_3 = 2*x_2*(1-x_2) = 0.49999872$$
$$x_{25} = 2*x_{24}*(1-x_{24}) \sim 0.5$$
